# Train gradient boosted trees ML for fSCA prediction

- Purpose: Train a gradient boosted trees ML model for fractional snow covered area (fSCA) using GPUs
- Creator: Cade Trotter with input from Ryan Crumley and Katrina Bennett
- Created: 2025.06.09; Last Modification: 2025.10.14
- See also: chunkedML.py
- Note: we used AI to create portions of the code below 

### Import packages / libraries

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import os
import subprocess
import dask
import dask.array as da
import json

dask.config.set(array__slicing__split_large_chunks=True)

print("All modules imported successfully :)")

## 1) User inputs and data paths

In [ ]:
# Change these paths for your purposes, the remainder of the code should not need to be changed

trainingWaterYears = [2011, 2012, 2013, 2015, 2016, 2017, 2018] #saving 2010, 2014, 2019 for validation. Oct 1 of WY -1 : Sep 30 of WY
n_chunks = len(trainingWaterYears) * 10 ### this sets how many chunks we will make. More chunks = less GPU memory at a given time
env = os.environ.copy()
env["CUDA_VISIBLE_DEVICES"] = "2"  # Or "" for all GPUs. This specifies which GPU to use

### Base Path:
dataDirectoryPath = ###put your base dir path here. If you do not wish to follow the dir structure below, paths will need to be
### ammended to reflect that. 

### Model Path:
base_run_path = dataDirectoryPath + "ML_model_dir/ML_model_version/" ###adjust depending on what you are doing

### Output paths
model_base = base_run_path + "model/" ###where the model will be stored during training and once done. Make this dir
npz_base = base_run_path +  "chunks/" ###where the IID training data chunks will be stored during training and once done. Make this dir
model_features_base = base_run_path + "featuresJsons/" ###where the features used in training will be stored during training and once done. Make this dir

### Save Paths (these should not need to be changed)
model_path = "model.pkl"
chunk_path = "chunk"
features_path = "feature_names.json"

### Establish File Paths to Important Datasets:

In [ ]:
### Note: If you do not wish to follow the dir structure below, paths will need to be
### ammended to reflect that. 

### The dims in these datasets should be in two categories:
### Static: (x,y)
### Dynamic: (time, x, y)
### Since they are being combined by coords in xarray the (time), x and y need to match in shape and value. Consider this in data prep

### Topographic (x,y):
elevPath = "goodDemXY.nc"
slopeAspectPath = "slopeAspectBinnedXY.nc"
latLongPath = "latLongXY.nc"

### Unique (time x,y):
dayOfWaterYearPaths = [dataDirectoryPath + "dowy/dayOfWaterYearGriddedSurface" + str(waterYear) + ".nc" for waterYear in trainingWaterYears]

### Forcing Variables (time x,y):
conusPaths = [dataDirectoryPath + "conusDaily/conusData" + str(waterYear) + "Complete.nc" for waterYear in trainingWaterYears]

### FSCA 5cm Threshold for a WY (time x,y)
fscaPaths = [dataDirectoryPath + "smFsca/smFsca_regridded_" + str(waterYear-1) + str(waterYear) + "XY.nc" for waterYear in trainingWaterYears]

### Precipitation Cummulative Summation (time x,y) (derived from CONUS: tiled, cummulative sum of RAINRATE starting on each water year)
precipPaths = [dataDirectoryPath + "conusDaily/conusData" + str(waterYear) + "Complete_cumsum_of_RAINRATE.nc" for waterYear in trainingWaterYears]

### Static Paths such as elev, slope, etc:
staticList = [elevPath, slopeAspectPath, latLongPath]
staticPaths = [dataDirectoryPath + "topo/" + path for path in staticList]

### Inspect the paths (if desired, uncomment)

In [ ]:
# staticPaths

In [ ]:
# conusPaths + fscaPaths + dayOfWaterYearPaths + precipPaths

### Establish the dynamic and static predictor variables, and the target variable for the ML

In [ ]:
# Define the dynamic or static predictor variables

# The dynamic variables are those that change with each timestep
dynamic_vars = ['T2D', 'LWDOWN', 'SWDOWN', 'U2D', 'V2D', 'day_of_year', 'accumulated_RAINRATE']    # Must be (time, x, y)

# The static variables are those that remain the same with each timestep (usually physiographic or geographic)
static_vars = ['HGT', 'slope', "binned_aspect", "lat", "lon"]   # Must be (x, y)

# The target variable
target_var = 'fSCA' # Must be (time, x, y)

# Load Datasets Lazily 
ds_dyn = xr.open_mfdataset(conusPaths + fscaPaths + dayOfWaterYearPaths + precipPaths, combine='by_coords', parallel=True, chunks={'time': 100})
ds_static = xr.open_mfdataset(staticPaths, combine='by_coords', parallel=True)  # elevation, slope etc.

### Fix some coords issues:

In [ ]:
ds_dyn = ds_dyn.reset_coords(["lat", "lon"], drop=False)
ds_dyn['lat'] = ds_dyn['lat'].transpose("x", "y")
ds_dyn['lon'] = ds_dyn['lon'].transpose("x", "y")

### Expand Dims and Combine:

In [ ]:
# Expand static variables across time 
ds_static_exp = ds_static.expand_dims(time=ds_dyn.time)

# Merge dynamic + static + target 
ds_combined = xr.merge([ds_dyn[dynamic_vars + [target_var]], ds_static_exp[static_vars]])

## 2) Reshape predictor variables and target variables for the ML

In [ ]:
X_all = ds_combined[dynamic_vars + static_vars].to_array().transpose(...).data.astype(np.float32)
y_all = ds_combined[target_var].data.astype(np.float32)

### Keep the variable labels for importance metrics in the model:

In [ ]:
feature_names = list(ds_combined[dynamic_vars + static_vars].to_array().coords['variable'].values)

with open(model_features_base + features_path, "w") as f:
    json.dump([str(f) for f in feature_names], f)

In [ ]:
print("Total samples:", X_all.shape)
print("Total samples:", y_all.shape)

### Transpose and flatten data for ML

In [ ]:
# Transpose X_all from (vars, time, x, y) → (time, x, y, vars)
X_all = X_all.transpose(1, 2, 3, 0)

# Transpose y_all to match X_all's spatial layout
y_all = y_all.transpose(0, 2, 1) 

# Flatten both
X_all = X_all.reshape(-1, 12)  
y_all = y_all.reshape(-1)       

#fix issue with NaNs in target data:
# expect that this will take a good while as the data is going from a lazy format to more eager when we compute and move it to Numpy format
mask = da.isfinite(y_all).compute()  # make the mask NumPy
X_all = X_all.compute()[mask]
y_all = y_all.compute()[mask]

###here we are expecting the data to be of shape (time*x*y, vars)
print("shape(X_all): ", np.shape(X_all))
print("shape(y_all): ", np.shape(y_all))

## 3) Make the data I.I.D. (random) and establish predictors & target

In [ ]:
# Independently and identically distributed = IID

# Create a list of indices that are random
n_samples = X_all.shape[0]
chunk_size = n_samples // n_chunks

# Reproducible shuffle
np.random.seed(42)
shuffled_indices = np.random.permutation(n_samples)

# Distribute random indices across chunks equally
chunk_indices = np.array_split(shuffled_indices, n_chunks)

# Define feature and target variables
predictor_vars = dynamic_vars + static_vars
target_var = target_var

## 4) Train using chunkedML.py:

In [ ]:
### we made a set of chunks to reduce GPU memory used at any given time. We are iterating over them here and training the model with them
for i, idx in enumerate(chunk_indices):
    print(f" Processing chunk {i}...")

    ### quick check
    if len(idx) == 0:
        print(f"Skipping chunk {i}: no samples")
        continue
    
    # proceed with training...
    x_path = f"{npz_base + chunk_path}_{i}_X.npy"
    y_path = f"{npz_base + chunk_path}_{i}_y.npy"

    if os.path.exists(x_path) and os.path.exists(y_path):
        print(f" Chunk {i} already saved — skipping recomputation.")
    else:
        idx_sorted = np.sort(idx)
        inv = np.argsort(np.argsort(idx))
    
        X_sorted = X_all[idx_sorted]
        y_sorted = y_all[idx_sorted]
    
        # Restore original shuffle order
        X_chunk = X_sorted[inv].reshape(-1, X_all.shape[1])  # shape (n_samples, n_features)
        y_chunk = y_sorted[inv].reshape(-1)  # shape (n_samples,)
    
        # Save to .npy files
        np.save(x_path, X_chunk)
        np.save(y_path, y_chunk)
        print(f"💾 Saved chunk {i} to disk.")

    # Build command to overwrite the same model file
    cmd = [
        "python", "/path/to/python/script/chunkedML.py",
        "--X", x_path,
        "--y", y_path,
        "--model-out", model_base + model_path,
        "--n-estimators", "50",
        "--device", "gpu",
        "--feature_names", model_features_base + features_path
    ]

    if os.path.exists(model_base + model_path):
        cmd += ["--model-in", model_base + model_path]

    print(f" Training model on chunk {i}...")
    result = subprocess.run(cmd, env=env, text=True)

    print(result.stdout)
    if result.returncode != 0:
        print(result.stderr)